

---



---

---


**Перед запуском кода загрузите файлы, из которых собираем БД формат файлов .txt**





---





In [31]:
### Установка фреймворков
!pip install -U langchain # общий фреймворк
!pip install langchain_community # для работы с загрузчиком документов
!pip install -U sentence-transformers # для
!pip install -U langchain-chroma # для создание БД

In [32]:
# Загрузчик документов
from langchain.document_loaders import DirectoryLoader, TextLoader

# в loader указываем путь в папку с базой знаний
# path - укажите путь к ПАПКЕ с файлами для Базы знаний ()
# glob - файлы какого формата будут учитываться, тут ".txt" формат
# с другими форматами не тестил

loader = DirectoryLoader(path='/content', glob = './*.txt', loader_cls= TextLoader)
documents = loader.load()


#  Ссылка на сайт для подробного понимания DirectoryLoader (зажми 'Ctrl' и нажми на ссылку)
#  https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.directory.DirectoryLoader.html

In [33]:
# Разбинеие на чанки
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size - регулирует размер чанков
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 200)
documents = text_splitter.split_documents(documents)

In [34]:
# Эмбеддинг
from langchain.embeddings import HuggingFaceBgeEmbeddings

# берём модель для эмбединга с Hugging Face
model_name = 'BAAI/bge-small-en-v1.5'
encode_kwargs = {'normalize_embeddings': True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    # вместо 'cpu' можно указать 'cuda' для более быстрой работы
    encode_kwargs=encode_kwargs
)

In [35]:
# Создание Базы знаний через Chroma (без сохранения Базы знаний)
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(documents, bge_embeddings)

In [36]:
#  Ретривер
# as_retriever извлекает из векторной БД чанки
retriever = vectordb.as_retriever()

# response имеет вид списка, у него буквально тип данных 'list'
response = retriever.invoke("where is human?")

# используем response[0], потому что response - список,
#  а к списку не может быть применён метод page_content
# метод page_content выводит ответ в человечном виде
# о да, обязательно попробуй запустить без page_content)
print(response[0].page_content)

In [ ]:
# И да доп. инфу и подробности можно искать через документацию langchain
# мужики, там есть поиск в правой верхнем углу (на сайте langchain)
# Вот ссылка (зажми 'Ctrl' и нажми на ссылку)
# https://python.langchain.com/docs/introduction/
